In [1]:
! pip install --upgrade kaggle

Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)


In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"moaz47","key":"df155534707966a9ddfe55b1c48b1539"}'}

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d abhranta/brain-tumor-detection-mri

 89% 57.0M/64.3M [00:01<00:00, 44.2MB/s]
100% 64.3M/64.3M [00:01<00:00, 44.4MB/s]


In [5]:
! mkdir brain_tumor

! unzip /content/brain-tumor-detection-mri.zip -d brain_tumor

Archive:  /content/brain-tumor-detection-mri.zip
  inflating: brain_tumor/Brain_Tumor_Detection/no/No12.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/No13.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/No14.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/No15.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/No17.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/No18.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/No19.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/no0.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/no1.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/no10.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/no100.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/no1000.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/no1001.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/no1002.jpg  
  inflating: brain_tumor/Brain_Tumor_Detection/no/no1003.jpg  
  inflating: brain_tumor/Brai

# Import Libraries

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

import cv2
import os

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import ipywidgets
from ipywidgets import widget
import shutil

In [7]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

# Create directoties for respective classes

In [8]:
_train = '/content/brain_tumor/Brain_Tumor_Detection'
_test = '/content/brain_tumor/Brain_Tumor_Detection/pred'


In [9]:
print(len(os.listdir(_train)))
print(len(os.listdir(_test)))

3
60


In [10]:
print(len(os.listdir(_train+'/no')))
print(len(os.listdir(_train+'/yes')))
print(len(os.listdir(_test)))

1500
1500
60


In [11]:
_no = os.listdir(_train+'/no')
_yes = os.listdir(_train+'/yes')
_pred = os.listdir(_test)

In [12]:
df1 = pd.DataFrame({'name': _no, 'type': 'no'})
df1['name'] = df1['name'].apply(lambda x: str(_train) + '/no/'+ x)

df2 = pd.DataFrame({'name': _yes, 'type': 'yes'})
df2['name'] = df2['name'].apply(lambda x: str(_train) + '/yes/'+ x)

train = pd.concat([df1, df2])
train.head()

,name,type
0,/content/brain_tumor/Brain_Tumor_Detection/no/...,no
1,/content/brain_tumor/Brain_Tumor_Detection/no/...,no
2,/content/brain_tumor/Brain_Tumor_Detection/no/...,no
3,/content/brain_tumor/Brain_Tumor_Detection/no/...,no
4,/content/brain_tumor/Brain_Tumor_Detection/no/...,no


In [13]:
test = pd.DataFrame(_pred, columns=['name'])
test['name'] = test['name'].apply(lambda x: str(_test) + '/' + x)
test.head()

,name
0,/content/brain_tumor/Brain_Tumor_Detection/pre...
1,/content/brain_tumor/Brain_Tumor_Detection/pre...
2,/content/brain_tumor/Brain_Tumor_Detection/pre...
3,/content/brain_tumor/Brain_Tumor_Detection/pre...
4,/content/brain_tumor/Brain_Tumor_Detection/pre...


# Copy files from source to destination

In [14]:
!mkdir train
!mkdir test

In [15]:
os.chdir('/content/train')

In [16]:
os.listdir(_train)

['yes', 'pred', 'no']

In [17]:
list_type = os.listdir(_train)
list_type.remove('pred')
list_type

['yes', 'no']

In [18]:
for i in list_type:
  os.mkdir(i)

In [19]:
destination = '/content/train'
count = 0
for i in list_type:
  li = []
  new_df = train[train['type']==i]
  li = new_df['name'].tolist()
  for j in li:
    shutil.move(j, destination+'/'+i)
    count = count + 1
  print(count)

1500
3000


In [20]:
os.chdir('/content/test')

In [21]:
"""
destination = '/content/test'
count = 0
for i in list_type:
  li = []
  new_df = test[test['type']==i]
  li = new_df['name'].tolist()
  for j in li:
    shutil.move(j, destination+'/'+i)
    count = count + 1
  print(count)
"""


destination = '/content/test'
count = 0
li = []
li = test['name'].tolist()
for j in li:
  shutil.move(j, destination)
  count = count + 1
print(count)

60


In [22]:
os.chdir('/content')

# Modifying train and test dataframes

In [23]:
train['name'] = train['name'].apply(lambda x: x.replace(_train + '/yes', "/content/train/yes"))
train['name'] = train['name'].apply(lambda x: x.replace(_train + '/no', "/content/train/no"))
train.head()

,name,type
0,/content/train/no/no32.jpg,no
1,/content/train/no/no853.jpg,no
2,/content/train/no/no1490.jpg,no
3,/content/train/no/no465.jpg,no
4,/content/train/no/no431.jpg,no


In [24]:
test['name'] = test['name'].apply(lambda x: x.replace(_test, "/content/test"))
test.head()

,name
0,/content/test/pred33.jpg
1,/content/test/pred1.jpg
2,/content/test/pred43.jpg
3,/content/test/pred34.jpg
4,/content/test/pred38.jpg


# Model development

In [25]:
# Fixed for our dataset
NUM_CLASSES = 2

# Fixed for our images
CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 7
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 32
BATCH_SIZE_VALIDATION = 32

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [26]:
model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights="imagenet"))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

94773248/94765736 [==============================] - 1s 0us/step


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [28]:
model.compile(optimizer = 'adam', loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [29]:
train_dir = '/content/train/'
valid_dir = '/content/test/'

In [30]:
image_size = IMAGE_RESIZE

# preprocessing_function is applied on each image but only after re-sizing & augmentation (resize => augment => pre-process)
# Each of the keras.application.resnet* preprocess_input MOSTLY mean BATCH NORMALIZATION (applied on each batch) stabilize the inputs to nonlinear activation functions
# Batch Normalization helps in faster convergence
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input, validation_split=0.25)

# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to target height and width.
        target_size = (image_size, image_size),
        batch_size = BATCH_SIZE_TRAINING,
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical',
        subset = 'training')

validation_data = train_datagen.flow_from_directory(
         train_dir,
         batch_size = BATCH_SIZE_VALIDATION,
         class_mode='categorical',
         subset='validation')

Found 2250 images belonging to 2 classes.
Found 750 images belonging to 2 classes.


In [31]:
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = './best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [32]:
fit_history = model.fit_generator(
        train_generator,
        epochs = NUM_EPOCHS,
        validation_data = validation_data,
        callbacks = [cb_checkpointer, cb_early_stopper]
)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/7
71/71 [==============================] - 51s 222ms/step - loss: 0.5910 - accuracy: 0.6939 - val_loss: 0.2388 - val_accuracy: 0.9120
Epoch 2/7
71/71 [==============================] - 13s 184ms/step - loss: 0.1873 - accuracy: 0.9413 - val_loss: 0.1881 - val_accuracy: 0.9413
Epoch 3/7
71/71 [==============================] - 13s 184ms/step - loss: 0.1623 - accuracy: 0.9429 - val_loss: 0.1700 - val_accuracy: 0.9467
Epoch 4/7
71/71 [==============================] - 13s 185ms/step - loss: 0.1325 - accuracy: 0.9551 - val_loss: 0.1613 - val_accuracy: 0.9493
Epoch 5/7
71/71 [==============================] - 13s 186ms/step - loss: 0.1018 - accuracy: 0.9685 - val_loss: 0.1569 - val_accuracy: 0.9547
Epoch 6/7
71/71 [==============================] - 13s 185ms/step - loss: 0.0947 - accuracy: 0.9736 - val_loss: 0.1222 - val_accuracy: 0.9640
Epoch 7/7
71/71 [==============================] - 13s 186ms/step - loss: 0.0700 - accuracy: 0.9851 - val_loss: 0.1348 - val_accuracy: 0.9507


In [ ]:
model.save_weights('/content/brain_tumor.hdf5')

In [ ]:
model.load_weights("./best.hdf5")

In [ ]:
model.save('/content/brain_tumor_resnet.h5')